In [21]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import zipfile
import math
import kaggle
import shutil
import glob
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from PIL import Image

In [2]:
print(tf.__version__)

2.17.0-dev20240310


In [ ]:
!kaggle datasets download -d preetviradiya/brian-tumor-dataset

In [12]:
with zipfile.ZipFile('brian-tumor-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall()


In [65]:
ROOT_DIR = "Brain Tumor Data Set/"
number_of_images = {}

In [66]:
for i in os.listdir(ROOT_DIR):
    if i.startswith("."):
        continue
    number_of_images[i] = len(os.listdir(os.path.join(ROOT_DIR,i)))

In [67]:
number_of_images

{'Brain Tumor': 2514, 'Healthy': 2087}

In [49]:
filetypes = {}
for k in number_of_images.keys():
    filetypes[k] = set()
    for i in os.listdir(os.path.join(ROOT_DIR,k)):
        filetypes[k].add(i.split(".")[-1])

In [50]:
filetypes

{'Brain Tumor': {'JPG', 'ipynb_checkpoints', 'jpg', 'png', 'tif'},
 'Healthy': {'JPG', 'jpeg', 'jpg', 'png'}}

In [48]:
!rm -rf Brain\ Tumor\ Data\ Set/'Brain Tumor'/ipynb_checkpoints

# Data Splitting for Training, Validation and Testing

- ```train``` : Directory which will contain the training images (70% of data)
- ```val``` : Directory which will contain  the validation images (15% of data)
- ```test``` : Directory which will contain the testing images (15% of data)

In [59]:
os.listdir("data/train/Brain Tumor")

[]

In [51]:
if not os.path.exists("data/train"):

    os.mkdir("data/train")

    for dir in os.listdir(ROOT_DIR):
        os.makedirs(f"data/train/{dir}")

        for img in np.random.choice(
            a = os.listdir(os.path.join(ROOT_DIR,dir))
            size = int(math.floor(0.7 * number_of_images[dir])),
            replace=False
        ):
        
    


    
else:
    print("Train exists")

Train does not exists
